In [ ]:
!pip install espn-api

In [ ]:
!pip install -U -q google-generativeai google-search-results
import os
import json
from google.colab import userdata
import google.generativeai as genai
from serpapi import GoogleSearch  # For Google Search integration
from IPython.display import display, HTML, Markdown

  Preparing metadata (setup.py) ... done


In [ ]:
# Google Generative AI API Key
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

# SerpApi API Key (for Google Search)
os.environ['SERPAPI_API_KEY'] = userdata.get('SERPAPI_API_KEY')

In [ ]:
def show_json(obj):
  print(json.dumps(obj.model_dump(exclude_none=True), indent=2))

def show_parts(r):
  parts = r.candidates[0].content.parts
  if parts is None:
    finish_reason = r.candidates[0].finish_reason
    print(f'{finish_reason=}')
    return
  for part in r.candidates[0].content.parts:
    if part.text:
      display(Markdown(part.text))
    elif part.executable_code:
      display(Markdown(f'```python\n{part.executable_code.code}\n```'))
    else:
      show_json(part)

  grounding_metadata = r.candidates[0].grounding_metadata
  if grounding_metadata and grounding_metadata.search_entry_point:
    display(HTML(grounding_metadata.search_entry_point.rendered_content))

In [ ]:
def google_search(query):
    """
    Performs a Google search using the SerpApi and returns a summarized result.
    """
    search = GoogleSearch({
        "q": query,
        "api_key": os.environ['SERPAPI_API_KEY']
    })
    results = search.get_dict()

    # Extract relevant information from the search results (customize as needed)
    summarized_results = []
    if 'organic_results' in results:
        for result in results['organic_results']:
            summarized_results.append({
                "title": result.get("title"),
                "snippet": result.get("snippet"),
                "link": result.get("link")
            })

    return summarized_results

In [ ]:
from espn_api.basketball import League

In [ ]:
# Initialize with your ESPN league ID and season year
# Replace with your league_id, espn_s2, and swid cookies
league_id = 1042708752
year = 2025
espn_s2 = 'AEAV%2F%2F33x5%2B8Q%2B6hO5yv58cZyYa%2BvcHcGSWQ9aGzAj9RlKLES3Pa6BVUkkPQoxDJYNb%2FbuLJLRKlMANMBcB5TO1DsQbowxzSepNgEoY3tzY%2BekTwiwjKHErVYjZRNVWOmT2lqqOmFgIBZxVtiiMCULOituOQBd6k4sim2VyLONkFQmTQTnDMxEP55dd1AUGuUOqYOiw%2FNIMWZKxJNhQ6lT9dBsjsEFVgFJyPa2lDPHMKUcCJJBbOxEsXEHrHXPfaO1%2BWdDqbS3RuAnxSIpUvZ8HqR6XrHmlV1ZoZwffriZepZw%3D%3D'
swid = '{70899C9A-651C-4224-9DDE-8A7C3E6D119C}'

In [ ]:
# Create league instance
league = League(league_id=league_id, year=year, espn_s2=espn_s2, swid=swid)

# Iterate through teams and print player data
for team in league.teams:
  print(f"Team: {team.team_name}")
  for player in team.roster:
    print(f"{player.name} - {player.position} - {player.total_points} points")

Team: Poole Party 
Nikola Jokic - C - 0 points
Jalen Brunson - PG - 0 points
Kristaps Porzingis - C - 0 points
Jamal Murray - PG - 0 points
Jordan Poole - SG - 0 points
Zach Edey - C - 0 points
Russell Westbrook - PG - 0 points
Chris Paul - PG - 0 points
Jonathan Kuminga - PF - 0 points
Michael Porter Jr. - SF - 0 points
Trey Murphy III - SF - 0 points
Peyton Watson - SF - 0 points
Lonzo Ball - PG - 0 points
Keon Ellis - SG - 0 points
Team: PG 13
Domantas Sabonis - C - 0 points
LeBron James - SF - 0 points
Tyrese Maxey - PG - 0 points
Jimmy Butler - SF - 0 points
Pascal Siakam - PF - 0 points
Jrue Holiday - PG - 0 points
Jalen Duren - C - 0 points
John Collins - PF - 0 points
Paul George - SF - 0 points
Aaron Gordon - PF - 0 points
Khris Middleton - SF - 0 points
Jordan Clarkson - SG - 0 points
Marcus Smart - SG - 0 points
Terry Rozier - PG - 0 points
Team: Arjun's Astounding Team
Victor Wembanyama - C - 0 points
Ja Morant - PG - 0 points
Devin Booker - PG - 0 points
Nikola Vucevic - C

In [ ]:
def get_fantasy_report(roster, opponent_roster, current_standings, league_settings):
    """
    Generates a fantasy basketball report with injury updates, news, trade, and waiver wire recommendations.
    Now utilizes Google Search to gather the latest information. Under no circustances should you hallucinate or give invalid citations. Double check your work and cite all your sources. You should have information up-to-date with January 7th 2025
    """
    model = genai.GenerativeModel(MODEL)  # Initialize the model here

    # 1. Gather Information Using Google Search:
    player_search_results = {}
    for player in roster:
        search_query = f"{player} NBA injury news"  # Customize your search queries
        player_search_results[player] = google_search(search_query)

    opponent_search_results = {}
    for player in opponent_roster:
        search_query = f"{player} NBA injury news"
        opponent_search_results[player] = google_search(search_query)

    # 2. Build the Prompt with Search Results:
    prompt = f"""
    You are an expert NBA fantasy basketball analyst. Under no circustances should you hallucinate or give invalid citations. Double check your work and cite all your sources. You should have information up-to-date with January 7th 2025. Provide a comprehensive report for my head-to-head matchup this week. Only use realtime, latest data, that is data in the last 24 hours, from reputable sources like ESPN.com, yahoosports.com and nbcsports.com, double check your work, do not hallucinate answers, and give citations for your findings.

    My Roster: {', '.join(roster)}
    Opponent's Roster: {', '.join(opponent_roster)}
    Current Standings: {current_standings}
    League Settings: {league_settings}

    The head to head categories are fg%, ft%, 3pm, rebounds, assists, steals, blocks, and points. One must win a majority of the 8 categories to win a particular week.

    Here's the latest information I found from Google searches for each player on my roster and my opponent's roster:

    """

    for player, results in player_search_results.items():
        prompt += f"\n**{player}:**\n"
        if results:
            for result in results:
                prompt += f"- {result['title']}: {result['snippet']} ({result['link']})\n"
        else:
            prompt += "- No recent news found.\n"

    for player, results in opponent_search_results.items():
        prompt += f"\n**Opponent's {player}:**\n"
        if results:
            for result in results:
                prompt += f"- {result['title']}: {result['snippet']} ({result['link']})\n"
        else:
            prompt += "- No recent news found.\n"

    prompt += """
    1. Injury Report and Latest News:
       - Provide the latest injury updates and relevant news for each player on my roster. This is data within Jan 1st 2025 to Jan 7th 2025.

    2. Trade Recommendations:
       - Analyze my roster and my opponent's roster.
       - Suggest potential trades that would improve my team's chances of winning this week and in the long run.
       - Consider the head-to-head categories and player performance.
       - Specifically state players I should trade away, players I should target, and potential trade partners.

    3. Waiver Wire Recommendations:
       - Based on my current roster, the head-to-head categories, and the league settings (especially waiver wire order), recommend players to pick up from the waiver wire.
       - Explain why each player would be a good addition.
       - Assume top free agents are available.

    4. Matchup specific advice:
        - Given my roster and my opponents roster, what are the key categories I should focus on this week and what advice do you have to secure a win?

    Generate a detailed and insightful report to help me win my matchup!
    """

    # 3. Generate the Report Using the Gemini Model:
    response = model.generate_content(prompt)
    return response.text

In [ ]:
# --- Example Usage (Replace with your actual data) ---

'''
my_roster = [
    "Stephen Curry",
    "Grayson Allen",
    "Haywood Highsmith",
    "Evan Mobley",
    "RJ Barrett",
    "Myles Turner",
    "Christian Braun",
    "Mikal Bridges",
    "Deandre Ayton",
    "Carlton Carrington",
    "Jusuf Nurkic",
    "Kelley Oubre Jr.",
    "Jalen Suggs"
    "Brandon Ingram (IR)",
]  # Replace with your players

'''
my_roster = []
opponent1_roster = []

for team in league.teams:
  if(team.team_name == "Greatest  Of All Time"):
    for player in team.roster:
      my_roster.append(player.name)
  if(team.team_name == "Lebrons fourskin"):
    for player in team.roster:
      opponent1_roster.append(player.name)
"""
opponent1_roster = [
    "Josh Giddey",
    "Dejounte Murray",
    "Luka Doncic",
    "Kawhi Leonard",
    "Jaren Jackson Jr.",
    "Fred VanVleet",
    "De'Aaron Fox",
    "Mikal Bridges",
    "Kristaps Porzingis",
    "Myles Turner",
    "Jrue Holiday",
    "Clint Capela",
    "Draymond Green"
]  # Replace with your opponent's players
"""

my_standings = {"wins": 4, "losses": 6, "ties": 1}
my_league_settings = {"roster_size": 15, "waiver_order": 2}

report = get_fantasy_report(
    my_roster, opponent1_roster, my_standings, my_league_settings
)
print(report)

Okay, here's a comprehensive fantasy basketball matchup report based on the latest available information (as of January 7, 2025) that you provided. I will double-check information for accuracy and provide citations.

**1. Injury Report and Latest News (Jan 1st 2025 - Jan 7th 2025):**

**Your Roster:**

*   **Stephen Curry (GSW):**  He was ruled out Saturday for injury management with bilateral knee soreness but is **NOT** on the injury report for Sunday's game, meaning he is available to play. (Foxsports.com)[https://www.foxsports.com/nba/stephen-curry-player] It is important to monitor his injury status daily due to the nature of the injury. 
*   **RJ Barrett (TOR):**  Returned from illness on Monday after missing several games. He is expected to have a minutes restriction in the first game back. (Fantasypros.com) [https://www.fantasypros.com/nba/news/492044/rj-barrett-illness-available-monday.php]
*    **Myles Turner (IND):** He is questionable for Thursday's game due to a right obli